In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os

# Step 1: 掛載 Google 雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

# Step 2: 載入現有的 .keras 模型
model_path = "/content/drive/My Drive/model_epoch_20.keras"  # 替換為您的模型路徑
model = load_model(model_path)

# Step 3: 解凍部分層進行微調
# 讓模型基礎部分（如 ResNet50）可訓練
base_model = model.layers[0]  # 假設 ResNet50 是模型的第一層
base_model.trainable = True  # 解凍整個基礎模型

# 鎖定前面的層，只解凍後幾層
for layer in base_model.layers[:-50]:  # 解凍最後 50 層
    layer.trainable = False

# Step 4: 重新編譯模型（使用較低學習率）
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
# Step 6: 準備數據集
train_path = "/content/drive/MyDrive/Colab Notebooks/dataset/flags/train"  # 替換為您的訓練集路徑
test_path = "/content/drive/MyDrive/Colab Notebooks/dataset/flags/test"   # 替換為您的測試集路徑
image_size = (224, 224)
batch_size = 32

# 載入數據集
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=image_size,
    batch_size=batch_size
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=image_size,
    batch_size=batch_size
)

# 數據增強與正規化
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

def preprocess(image, label):
    image = data_augmentation(image)
    image = tf.cast(image, tf.float32) / 255.0  # 正規化到 [0, 1]
    return image, label

train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Step 6: 訓練回調
checkpoint_dir = "/content/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}.keras")
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    save_best_only=True,
    monitor="val_loss",
    verbose=1
)

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6)

# Step 7: 繼續訓練模型
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,
    callbacks=[model_checkpoint, early_stopping, reduce_lr]
)

# Step 8: 保存最終模型
final_model_path = "/content/drive/My Drive/fine_tuned_model.keras"
model.save(final_model_path)
print(f"模型已保存到: {final_model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 19716 files belonging to 249 classes.
Found 5154 files belonging to 249 classes.
Epoch 1/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5465 - loss: 4.0356